In [18]:
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=np.nan)
model = KeyedVectors.load_word2vec_format('/Word_sense_all_data/bio_nlp_vec/PubMed-shuffle-win-30.bin', binary=True)

In [2]:
pd.options.display.max_rows = 999

In [3]:
def sentence_vector(sentence):
    word_list = sentence.split()
    #stemmer = SnowballStemmer("english", ignore_stopwords=True)
    #word_list = [stemmer.stem(x) for x in word_list]
    word_list = [word for word in word_list if word not in stop_words]
    
    word_vectors = []
    for x in word_list:
        try:
            word_vectors.append(model[x])
        except KeyError:
            None    
    return sum(word_vectors)/len(word_vectors)
def vector_breakage(sentence):
    word_list = sentence.split()
    #stemmer = SnowballStemmer("english", ignore_stopwords=True)
    #word_list = [stemmer.stem(x) for x in word_list]
    word_list = [word for word in word_list if word not in stop_words]
    word_vectors_list = []
    for x in word_list:
        try:
            if len(model[x])==200:
                word_vectors_list.append(x)
        except:
            None
        else:
            None
    return word_vectors_list

In [4]:
stop_words = "disease, diseases, disorder, symptom, symptoms, drug, drugs, problems, problem,prob, probs, med, meds,\
, pill, pills, medicine, medicines, medication, medications, treatment, treatments, caps, capsules, capsule,\
, tablet, tablets, tabs, doctor, dr, dr., doc, physician, physicians, test, tests, testing, specialist, specialists,\
, side-effect, side-effects, patient, patients, pharmaceutical, pharmaceuticals, pharma, diagnosis, diagnose, diagnosed, exam,\
, challenge, device, condition, conditions, suffer, suffering ,suffered, feel, feeling, prescription, prescribe,\
, prescribed, over-the-counter, a, about, above, after, again, against, all, am, an, and, any, are, aren't, as, at, be, because, been, before,\
, being, below, between, both, but, by, can, can't, cannot, could, couldn't, did, didn't, do, does, doesn't,\
, doing, don't, down, during, each, few, for, from, further, had, hadn't, has, hasn't, have, haven't, having, he,\
, he'd, he'll, he's, her, here, here's, hers, herself, him, himself, his, how, how's, i, i'd, i'll, i'm, i've, if, in, into,\
, is, isn't, it, it's, its, itself, let's, me, more, most, mustn't, my, myself, no, nor, not, of, off, on, once, only, or,\
, other, ought, our, ours , ourselves, out, over, own, same, shan't, she, she'd, she'll, she's, should, shouldn't,\
, so, some, such, than, that, that's, the, their, theirs, them, themselves, then, there, there's, these, they,\
, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, very, was, wasn't, we, we'd,\
, we'll, we're, we've, were, weren't, what, what's, when, when's, where, where's, which, while, who, who's,\
, whom, why, why's, with, won't, would, wouldn't, you, you'd, you'll, you're, you've, your, yours, yourself,\
, yourselves, n't, 're, 've, 'd, 's, 'll, 'm".replace(',','').split(' ')

In [10]:
df = pd.read_csv("/Word_sense_all_data/PatientNotes_acronym_longform.csv")
group_df = df[['abbreviation','long_form']]
group_df = group_df.groupby('abbreviation').agg(lambda x: set(x.tolist()))
group_df

,long_form
abbreviation,
AB,"{abortion, MISTAKE:abduction, arteriovenous:AV, arterial blood, antipyrine benzocaine, type A, type B, X-ray finding, atrioventricular:AV, blood group in ABO system, ankle-brachial, NAME, UNSURED SENSE}"
AC,"{acromioclavicular, angiotensin-converting enzyme:ACE, antecubital, acetate, adriamycin cyclophosphamide, alternating current, anticoagulation, assist control, before meals, (drug) AC, abdominal circumference}"
ALD,"{alanine aminotransferase:ALT, acetyl lysergic acid diethylamide, left anterior descending:LAD, ad lib on demand, adrenoleukodystrophy}"
AMA,"{against medical advice, advanced maternal age, antimitochondrial antibody}"
ASA,"{acetylsalicylic acid, American Society of Anesthesiologists, aminosalicylic acid}"
AV,"{atrioventricular, aortic valve, arteriovenous, UNSURED SENSE}"
AVR,"{aortic valve replacement, aortic valve regurgitation, aortic valve resistance, auditory brainstem response:ABR, rapid ventricular response:RVR, augmented voltage right arm, UNSURED SENSE}"
BAL,"{blood alcohol level, bronchoalveolar lavage}"
BK,"{BK (virus), below knee}"


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
np.set_printoptions(threshold=np.nan)

In [12]:
def find_acronym_full_form(sentence):
    df = pd.read_csv("/Word_sense_all_data/PatientNotes_acronym_longform.csv")
    acronym_list = list(df.abbreviation.unique())
    positive_acronym = []
    for word in sentence.split():
        if word in acronym_list:
            positive_acronym.append(word)
    positive_acronym_strength = {}
    for x in positive_acronym:
        positive_acronym_strength[x] = len(group_df.loc[x]['long_form'])
    #print(positive_acronym_strength)
    acronym_found = sorted(positive_acronym_strength, key=positive_acronym_strength.get, reverse=False)
    #print(acronym_found)
    
    #print([word for word in vector_breakage(sentence) if word not in acronym_found])
    sentence = ''.join([word for word in sentence if word not in acronym_found])
    for item in acronym_found:
        df_slice = df[df.abbreviation==item]
        y = df_slice.long_form
        X = list([sentence_vector(x) for x in df_slice.sentence])
        #X_train, X_test, y_train, y_test = train_test_split(X, y)
        clf = SVC(C=1.0, kernel='linear', degree=1,probability = True).fit(X, y)
        print(clf.predict_proba(sentence_vector(sentence).reshape(1,-1)))
        return clf.predict(sentence_vector(sentence).reshape(1, -1))[0]

In [15]:
find_acronym_full_form('MS is a disease in which the immune system eats away at the protective covering of nerves')

[[0.00311797 0.00217474 0.00310901 0.00893966 0.00230238 0.00343115
  0.00282467 0.0214154  0.95015775 0.00252728]]


'multiple sclerosis'

## Test

In [11]:
find_acronym_full_form('lung malignant CA')

[[0.01158447 0.00830349 0.02475965 0.9553524 ]]


'carbohydrate antigen'

In [153]:
print(model.most_similar(positive = 'effusion', topn = 6))

[('effusions', 0.8661322593688965), ('pleural', 0.7674405574798584), ('pericardial', 0.7196462154388428), ('pleuro-pericardial', 0.7094818353652954), ('Pleuropericardial', 0.7049542665481567), ('transudative', 0.7045389413833618)]
